# Baseline. Naive Bayes + Bag-of-Words

---

Классификация дихотомий MBTI с помощью Bag-of-Words + Multinomial Naive Bayes.

**Архитектура модели:**
- Multinomial Naive Bayes на Bag-of-Words n-граммах (1-2)
- 4 бинарных классификатора для дихотомий: I/E, N/S, T/F, J/P

**Метрики:** accuracy и F1 для каждой дихотомии + exact match

---

vers. 1.0.0


## 1. Импорт библиотек

Импорт pandas, numpy, sklearn, matplotlib, seaborn

In [ ]:
%pip install -q scikit-learn pandas numpy matplotlib seaborn

import os
import sys
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# Добавляем корень проекта, чтобы импортировать препроцессор
project_root = os.path.abspath('..')
sys.path.append(project_root)

from src.data.preprocessor import MBTIPostPreprocessor

print("✅ Библиотеки импортированы")


## 2. Загрузка и первичный анализ датасета

* Файл: `data/raw/mbti_dataset.csv`
* Колонки: `type`, `posts`
* Проверяем размер, распределение классов, пример поста


In [ ]:
# Загрузка датасета
DATA_PATH = '../data/raw/mbti_dataset.csv'

df = pd.read_csv(DATA_PATH)
print(f"📊 Размер: {df.shape}")
print(df.head(2))

# Распределение типов
plt.figure(figsize=(10,4))
ax = df['type'].value_counts().plot(kind='bar', color='steelblue')
plt.title('Распределение MBTI типов')
plt.xticks(rotation=45)
plt.show()

print("\nПример поста:\n", df['posts'].iloc[0][:300], '...')


## 3. Предобработка текста

Используем `MBTIPostPreprocessor` (тот же, что в LSTM-пайплайне): 
* lowercase, удаление URL/спецсимволов, стоп-слова, лемматизация.
* Преобразуем каждый пост в очищенную строку для Bag-of-Words.


In [ ]:
# Предобработка всех постов (~2-3 мин на CPU)
preprocessor = MBTIPostPreprocessor(
    lowercase=True,
    remove_urls=True,
    remove_special_chars=True,
    remove_stopwords=True,
    lemmatize=True
)

df['clean_text'] = df['posts'].apply(preprocessor.preprocess_to_string)
print(df[['posts', 'clean_text']].head(1).iloc[0].to_string())


## 4. Bag-of-Words векторизация

* `CountVectorizer(ngram_range=(1,2), max_features=10_000, stop_words='english')`
* Получаем матрицу признаков `X`.


In [ ]:
vectorizer = CountVectorizer(ngram_range=(1,2), max_features=10_000, stop_words='english')
X = vectorizer.fit_transform(df['clean_text'])
print(f"Матрица признаков: {X.shape}")


## 5. Кодирование меток и разбиение выборки

* Каждая дихотомия — независимая бинарная задача.
* Создаём `y_dict` с 4 бинарными векторами.
* Используем `train_test_split` (70/30) с `stratify` по полной MBTI-строке, чтобы сохранить баланс.


In [ ]:
# Кодирование дихотомий
DICHOTOMIES = [(0, 'IE'), (1, 'NS'), (2, 'TF'), (3, 'JP')]

y_dict = {}
for idx, name in DICHOTOMIES:
    y = np.where(df['type'].str.upper().str[idx] == name[1], 1, 0)
    y_dict[name] = y

# Train/Test split индексов (общий stratifiy)
train_idx, test_idx = train_test_split(
    np.arange(len(df)), test_size=0.3, stratify=df['type'], random_state=42)

X_train, X_test = X[train_idx], X[test_idx]
print("Train shape:", X_train.shape, " Test shape:", X_test.shape)


## 6. Обучение Multinomial Naive Bayes

* Обучаем 4 модели — по одной на дихотомию.
* Сохраняем метрики в `results`.


In [ ]:
models = {}
results = {}

for _, name in DICHOTOMIES:
    y_train = y_dict[name][train_idx]
    y_test = y_dict[name][test_idx]

    clf = MultinomialNB(alpha=1.0)
    clf.fit(X_train, y_train)
    models[name] = clf

    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary', zero_division=0)

    results[name] = {'acc': acc, 'prec': prec, 'rec': rec, 'f1': f1}
    print(f"{name}: Acc={acc:.3f} | F1={f1:.3f}")


## 7. Exact-match accuracy и сводная таблица

* Собираем предсказания всех 4 моделей.
* Считаем долю примеров, где угаданы все дихотомии сразу.
* Формируем DataFrame `results_df`.


In [ ]:
# Exact-match
pred_matrix = []
for _, name in DICHOTOMIES:
    y_pred = models[name].predict(X_test)
    pred_matrix.append(y_pred)

y_pred_all = np.vstack(pred_matrix).T  # shape (n_samples, 4)

y_test_all = np.vstack([y_dict[name][test_idx] for _, name in DICHOTOMIES]).T

exact_match = (y_pred_all == y_test_all).all(axis=1).mean()
print(f"\nExact-match accuracy (полный MBTI тип): {exact_match:.3f}")

# Таблица метрик
results_df = pd.DataFrame(results).T
results_df


## 8. Визуализация метрик

Bar-plot accuracy и F1 по дихотомиям + сравнение с exact-match.


In [ ]:
# Bar plot
fig, ax = plt.subplots(1, 2, figsize=(12,4))

results_df['acc'].plot(kind='bar', ax=ax[0], color='skyblue')
ax[0].set_title('Accuracy по дихотомиям')
ax[0].set_ylim(0,1)

results_df['f1'].plot(kind='bar', ax=ax[1], color='salmon')
ax[1].set_title('F1-score по дихотомиям')
ax[1].set_ylim(0,1)

plt.tight_layout()
plt.show()

print(f"Exact-match accuracy: {exact_match:.3f}")


## 9. Выводы

* Naive Bayes даёт базовую точность `~0.60` на лучших дихотомиях.
* Exact-match ниже (обычно ~0.25-0.30).
* БиLSTM из основного ноутбука ожидаемо превосходит NB на **+10-20 p.p.**

Дальнейшие шаги:
1. Добавить Logistic Regression + TF-IDF.
2. Попробовать DistilBERT для fair-сравнения.
3. Интегрировать все baseline-результаты в README / отчёт.
